# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [ ]:
ONLINE TRIP PLANNING ASSISTANT

In [ ]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""

In [ ]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

In [ ]:
system_message="You are the AI trip planner assistant your job is to analyse the user message and give clear and crisp response based on their queries."
system_message+="Only if they are asking you specifically for iternary for their trip destination for the required number of days provided by them in the user message. for example"
system_message+="user: I want a plan for Ooty trip for 3 days." 
system_message+="then you need to provide an clear simple iternary for OOty for 3 days. Like Day1 this place, this place Day 2 this place this place in a good format."
system_message+="Always be accurate if you don't know the answer say so."

In [ ]:
# This function looks rather simpler than the one from my video, because we're taking advantage of the latest Gradio updates

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages",js=force_dark_mode).launch()

In [ ]:
#let add mulitmodal feature image 
import base64
from io import BytesIO
from PIL import Image

In [ ]:
def artist(city):
    image_response = openai.images.generate(
            model="dall-e-3",
            prompt=f"An image representing a vacation in {city}, showing tourist spots and everything unique about {city}, in a vibrant pop-art style",
            size="1024x1024",
            n=1,
            response_format="b64_json",
        )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

In [ ]:
#now lets implement tools for checking the ticket price and for demo flight booking assitance

In [ ]:
# Add this import at the top
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.units import inch
from datetime import datetime
import io
import base64


In [ ]:
#for checking ticket price

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")


In [ ]:
#for flight booking
# Add the booking function
def book_flight(destination_city, travel_date, passenger_name="Traveler"):
    print(f"Tool book_flight called for {destination_city} on {travel_date}")
    
    # Generate a simple PDF ticket
    buffer = io.BytesIO()
    c = canvas.Canvas(buffer, pagesize=letter)
    width, height = letter
    
    # Draw ticket
    c.setFont("Helvetica-Bold", 24)
    c.drawString(2*inch, height - 2*inch, "FLIGHT TICKET")
    
    c.setFont("Helvetica", 12)
    c.drawString(1*inch, height - 3*inch, f"Passenger: {passenger_name}")
    c.drawString(1*inch, height - 3.5*inch, f"Destination: {destination_city.upper()}")
    c.drawString(1*inch, height - 4*inch, f"Travel Date: {travel_date}")
    c.drawString(1*inch, height - 4.5*inch, f"Booking Reference: BK{datetime.now().strftime('%Y%m%d%H%M%S')}")
    c.drawString(1*inch, height - 5*inch, f"Price: {ticket_prices.get(destination_city.lower(), 'N/A')}")
    c.drawString(1*inch, height - 6*inch, "Status: CONFIRMED ✓")
    
    c.save()
    
    pdf_bytes = buffer.getvalue()
    buffer.close()
    
    return base64.b64encode(pdf_bytes).decode(), destination_city, travel_date


In [ ]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

booking_function = {
    "name": "book_flight",
    "description": "Book a flight ticket to the destination city for a specific date. Call this when a customer wants to book a flight and provides a travel date.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The destination city for the flight",
            },
            "travel_date": {
                "type": "string",
                "description": "The date of travel in format YYYY-MM-DD or any date format provided by user",
            },
            "passenger_name": {
                "type": "string",
                "description": "The name of the passenger (optional)",
            }
        },
        "required": ["destination_city", "travel_date"],
        "additionalProperties": False
    }
}

In [ ]:
tools = [
    {"type": "function", "function": price_function},
    {"type": "function", "function": booking_function}
]

In [ ]:
# Update handle_tool_call to handle both tools
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    
    if tool_call.function.name == "get_ticket_price":
        city = arguments.get('destination_city')
        price = get_ticket_price(city)
        response = {
            "role": "tool",
            "content": json.dumps({"destination_city": city, "price": price}),
            "tool_call_id": tool_call.id
        }
        return response, city
    
    elif tool_call.function.name == "book_flight":
        city = arguments.get('destination_city')
        date = arguments.get('travel_date')
        passenger = arguments.get('passenger_name', 'Traveler')
        pdf_base64, city, date = book_flight(city, date, passenger)
        response = {
            "role": "tool",
            "content": json.dumps({
                "destination_city": city,
                "travel_date": date,
                "status": "confirmed",
                "ticket_pdf": pdf_base64
            }),
            "tool_call_id": tool_call.id
        }
        return response, city

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    
    pdf_file = None
    image = None
    
    if response.choices[0].finish_reason == "tool_calls":
        message_obj = response.choices[0].message
        tool_response, city = handle_tool_call(message_obj)
        
        # Check which tool was called
        if message_obj.tool_calls[0].function.name == "get_ticket_price":
            image = artist(city)
        
        elif message_obj.tool_calls[0].function.name == "book_flight":
            tool_content = json.loads(tool_response["content"])
            pdf_base64 = tool_content.get("ticket_pdf")
            
            if pdf_base64:
                # Decode and save PDF
                pdf_bytes = base64.b64decode(pdf_base64)
                pdf_file = f"ticket_{city}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pdf"
                with open(pdf_file, "wb") as f:
                    f.write(pdf_bytes)
        
        messages.append(message_obj)
        messages.append(tool_response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content, pdf_file, image

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("# 🌍 AI Trip Planner Assistant")
    
    with gr.Row():
        chatbot = gr.Chatbot(type="messages", height=400)
        image_output = gr.Image(height=400, interactive=False, show_label=False)
    
    with gr.Row():
        msg = gr.Textbox(label="Your message", placeholder="Ask about ticket prices or book a flight...", scale=4)
        send_btn = gr.Button("Send", scale=1)
    
    file_output = gr.File(label="📄 Download Your Ticket", visible=True)
    
    def respond(message, history):
        bot_message, pdf_path, image = chat(message, history)
        history.append({"role": "user", "content": message})
        history.append({"role": "assistant", "content": bot_message})
        return history, "", pdf_path, image
    
    msg.submit(respond, [msg, chatbot], [chatbot, msg, file_output, image_output])
    send_btn.click(respond, [msg, chatbot], [chatbot, msg, file_output, image_output])

demo.launch()